In [11]:
#Expense Tracker App
import sqlite3

# Database connection details
DATABASE_FILE = "expenses.db"

class Expense:
    def __init__(self, date, description, amount):
        self.date = date
        self.description = description
        self.amount = amount


class ExpenseTracker:
    def __init__(self):
        self.conn = self._create_connection()

    def _create_connection(self):
        """Creates a connection to the SQLite database."""
        try:
            conn = sqlite3.connect(DATABASE_FILE)
            print("Connected to SQLite database.")
            self._create_table(conn)  # Created table on first connection
            return conn
        except sqlite3.Error as e:
            print(f"Error connecting to database: {e}")
            return None

    def _create_table(self, conn):
        """Creates the 'expenses' table if it doesn't already exist."""
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS expenses (
                         date text,
                         description text,
                         amount real
                         )''')
        conn.commit()

    def add_expense(self, expense):
        """Adds an expense to the database."""
        cursor = self.conn.cursor()
        cursor.execute("INSERT INTO expenses (date, description, amount) VALUES (?, ?, ?)",
                       (expense.date, expense.description, expense.amount))
        self.conn.commit()
        print("Expense added successfully!")

    def remove_expense(self, index):
        """Removes an expense from the database based on index."""
        if 0 <= index < self.get_total_expenses():
            cursor = self.conn.cursor()
            cursor.execute("DELETE FROM expenses WHERE ROWID=?", (index + 1,))
            self.conn.commit()
            print("Expense removed!")
        else:
            print("Invalid expense index")

    def view_expenses(self):
        """Fetches and displays all expenses from the database."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM expenses")
        expenses = cursor.fetchall()

        if not expenses:
            print("No expenses found")
        else:
            print("\nExpense List:")
            for i, expense in enumerate(expenses, start=1):
                print(f"{i}. Date: {expense[0]}, Description: {expense[1]}, Amount: Rs{expense[2]:.2f}")

    def get_total_expenses(self):
        """Calculates and returns the total expense from the database."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT SUM(amount) FROM expenses")
        total = cursor.fetchone()[0]  # Get the first element from the returned tuple
        return total if total else 0  # Handle empty result

    def total_expense(self):
        """Prints the total expense."""
        total = self.get_total_expenses()
        print(f"\nTotal Expense: Rs{total:.2f}")

    def main(self):
        while True:
            print("\nWelcome to My Expense Tracker")
            print("Select an option:")
            print("1 - Add Expense")
            print("2 - Remove Expense")
            print("3 - View Expenses")
            print("4 - Total Expense")
            print("5 - Quit")

            choice = input("Enter your choice: ")

            if choice == "1":
                date = input("Enter the date (YYYY-MM-DD): ")
                description = input("Enter the description: ")
                amount = float(input("Enter the amount: "))
                expense = Expense(date, description, amount)
                self.add_expense(expense)

            elif choice == "2":
                index = int(input("Enter the index to remove: ")) - 1
                self.remove_expense(index)

            elif choice == "3":
                self.view_expenses()

            elif choice == "4":
                self.total_expense()

            elif choice == "5":
                print("Thanks for using the Expense Tracker!")
                self.conn.close()  # Closed the database connection on exit
                break

            else:
                print("Invalid choice. Please try again.")


if __name__ == "__main__":
    tracker = ExpenseTracker()
    tracker.main()


Connected to SQLite database.

Welcome to My Expense Tracker
Select an option:
1 - Add Expense
2 - Remove Expense
3 - View Expenses
4 - Total Expense
5 - Quit
Enter your choice: 5
Thanks for using the Expense Tracker!
